# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [1]:
import pandas as pd
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df_insurance = pd.read_csv(url)

In [3]:
pd.set_option('display.max_columns', None)

In [5]:
df_insurance.columns = df_insurance.columns.str.lower().str.replace(' ', '_')

In [ ]:
df_insurance.sample(10)

-  DataFrame que solo incluya a los clientes que cumplan con las dos condiciones, 'total_claim_amount'>1000 & response == Yes

In [7]:
df_1 = df_insurance[(df_insurance['total_claim_amount'] > 1000) & (df_insurance['response'] == 'Yes')]

In [ ]:
df_1.sample(10)

- Promedio de 'total_claim_amount' por tipo de póliza y género para los clientes que respondieron 'Yes'

- Pivot table que agrupa los datos de 'policy_type' y 'gender', luego calcular la media de la columna 'total_claim_amount' únicamente para los clientes con response igual a 'Yes'.

In [9]:
df_yes = df_insurance[df_insurance['response'] == 'Yes'] #  Solo los clientes que respondieron 'Yes' a la campaña

In [10]:
df_2 = pd.pivot_table(df_yes, values='total_claim_amount', index='policy_type', columns='gender', aggfunc='mean')

In [11]:
df_2 = df_2.round(2)

In [ ]:
df_2

- Ejercicio 3.

- Contar el número total de clientes que tienen pólizas en cada estado.
- Filtrar el resultado para incluir solo los estados donde hay más de 500 clientes.

In [18]:
df_clientes_por_estado = df_insurance['state'].value_counts()

In [19]:
df_estados_mas_500 = df_clientes_por_estado[df_clientes_por_estado > 500]

In [ ]:
df_estados_mas_500

- Ejercicio 4: valor máximo, mínimo, y mediana del customer_lifetime_value por nivel educativo y género.

In [ ]:
df_4 = pd.pivot_table(df_insurance, values='customer_lifetime_value', index=['education', 'gender'], aggfunc=[max, min, 'median'])

In [22]:
df_4 = df_4.round(2)

In [ ]:
df_4

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here